<a href="https://colab.research.google.com/github/singclare/DataAnalysis/blob/master/%EC%A2%85%ED%95%A9_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import imblearn
from imblearn import under_sampling
import random
import cv2
from tqdm import tqdm
from google.colab.patches import cv2_imshow

In [ ]:
path = "/content/drive/MyDrive/데이터셋"
os.chdir(path)
class_list = os.listdir(path)[1:21]

In [ ]:
class_list = os.listdir(path)[1:21]
classnumber = len(class_list)

#undersampling

In [ ]:
image_data = np.load('./data224_final cluster.npy')
class_data = np.load('./data label 224_final cluster.npy')

In [ ]:
data = image_data
class_ = class_data

In [ ]:
import imblearn.under_sampling.base
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [ ]:
class_data = pd.DataFrame(class_)
class_data = class_data.reset_index(drop = False)
class_data_x = pd.DataFrame(class_data['index'])
class_data = class_data.drop('index',axis=1)
# 추출한 클래스 데이터의 인덱스를 따로 저장해서 해당 인덱스의 그림 데이터를 불러오는 방식
class_resampled, index_resampled = RandomUnderSampler(sampling_strategy = 'majority', random_state=1234).fit_resample(class_data, class_data_x)

In [ ]:
index = index_resampled['index']
data_resampled = []

for i in index:
  data_resampled.append(data[i,:,:,:])

data_resampled = np.array(data_resampled)
# 언더샘플링 된 데이터는 data_resampled 와 class_resampled에 각각 저장됨

In [ ]:
image_data.shape

(23567, 224, 224, 3)

In [ ]:
data_resampled, label_resampled = RandomUnderSampler(sampling_strategy = 'majority', random_state=1234).fit_resample(image_data, label_)

NameError: ignored

# 기존 train-test split

stratified version

In [ ]:
import re

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from keras import layers
from keras import Input, Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, class_, stratify = class_)

In [ ]:
X_train = X_train / 255
X_test  = X_test / 255

In [ ]:
# 모델 구축
model = Sequential()
model.add(Convolution2D(112, 2, 2, padding = 'same',
    input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(224, 2, 2, padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(224, 2, 2))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(classnumber))
model.add(Activation('softmax'))

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
optimizer = keras.optimizers.Adam()
# compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])


# fit the model
history = model.fit(X_train, y_train, batch_size = 32, epochs=30)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)

#stratifiedfold

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_resampled, class_resampled, stratify = class_resampled)
xy = (X_train, X_test, y_train, y_test)

X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255

y_train = keras.utils.to_categorical(y_train, 20)
y_test = keras.utils.to_categorical(y_test, 20)

print('X_train shape:', X_train.shape)

image_data_index = data_resampled[:,0,0,0]
image_data_index = pd.DataFrame(image_data_index.reshape(-1,1))

X_train shape: (1815, 224, 224, 3)


In [ ]:
# 모델 구축
model = Sequential()
model.add(Convolution2D(112, 3, 3, padding = 'same',
    input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(224, 3, 3, padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(224, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(classnumber))
model.add(Activation('softmax'))

NameError: ignored

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
optimizer = keras.optimizers.Adam()
skf =StratifiedKFold(n_splits = 2, random_state = None)
cnt_iter = 0
Fold_accuracy = []
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])


for fold, (train_index, val_index) in enumerate(skf.split(image_data_index, class_resampled)):
  
  cnt_iter += 1

  X_train, X_val, y_train, y_val = data_resampled[train_index], data_resampled[val_index], class_resampled.loc[train_index], class_resampled.loc[val_index]

  y_train = keras.utils.to_categorical(y_train, 20)
  y_val = keras.utils.to_categorical(y_val, 20)

  history = model.fit(X_train, y_train, batch_size = 32, epochs=30)

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_val, y_val)
  print(f'{fold+1}번째 fold의 정확도는 {accuracy}입니다')
  Fold_accuracy.append(accuracy)

In [ ]:
np.mean(Fold_accuracy)

0.4388429671525955

In [ ]:
model.predict(X_test)

#autoencoder

In [ ]:
import torch
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras import backend as K

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
idg = ImageDataGenerator(rescale = 1./255, horizontal_flip = True)
idg2 = ImageDataGenerator(rescale = 1./255, )

NameError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, class_, train_size = 0.05, stratify = class_)

In [ ]:
input_img = keras.Input(shape=(224, 224, 3))
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2))(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2))(x)

x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2))(x)

x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), strides=(2,2))(x)
print(encoded.shape)

# (7, 7, 512)

x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(encoded)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
print(x.shape)

decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
print(decoded.shape)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

(None, 7, 7, 512)
(None, 224, 224, 64)
(None, 224, 224, 3)


In [ ]:
X_train.shape

(1178, 224, 224, 3)

# 64 하이퍼파라미터 조정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

import re
import imblearn
from imblearn import under_sampling

import random
import cv2
from tqdm import tqdm
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

import re

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from keras import layers
from keras import Input, Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
path = "/content/drive/MyDrive/데이터셋"
os.chdir(path)
class_list = os.listdir(path)[1:21]

In [ ]:
totaldata = pd.read_csv('under sample_cluster_image_path and label.csv', usecols=[1,2])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
idg = ImageDataGenerator(rescale = 1./255, horizontal_flip = True)
idg2 = ImageDataGenerator(rescale = 1./255, )

In [ ]:
totaldata = totaldata.astype({'label' : 'str'})
from sklearn.cluster import KMeans

from imblearn.under_sampling import *
for train_index, test_index in skf.split(totaldata, totaldata['label']):
 
    training_set = totaldata.iloc[train_index]
    validation_set = totaldata.iloc[test_index]

    undersample = RandomUnderSampler(sampling_strategy='majority')
    X_under, y_under = undersample.fit_resample(training_set, training_set['label'])
    X_under, y_under = undersample.fit_resample(X_under, y_under)
    X_under, y_under = undersample.fit_resample(X_under, y_under)
    X_under, y_under = undersample.fit_resample(X_under, y_under)

    resampled_training_set = X_under

    train_data_generator = idg.flow_from_dataframe(resampled_training_set, x_col = "path", y_col = "label", color_mode='rgb',
                                                   target_size = (64, 64), class_mode='categorical', batch_size=64, shuffle=True)
    valid_data_generator = idg.flow_from_dataframe(validation_set, x_col = "path", y_col = "label", color_mode='rgb',
                                                   target_size = (64, 64), class_mode='categorical', batch_size=64, shuffle=True)

In [ ]:
# 모델 구축
model = Sequential()
model.add(Convolution2D(32, 2, 2, padding = 'same',
    input_shape = (64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 2, 2, padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 2, 2))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(classnumber))
model.add(Activation('softmax'))

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
optimizer = keras.optimizers.Adam()
# compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])


# fit the model
history = model.fit_generator(train_data_generator, epochs=30)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
95/95 [==============================] - 1559s 16s/step - loss: 2.8775 - acc: 0.0992 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 2/30
68/95 [====================>.........] - ETA: 54s - loss: 2.8161 - acc: 0.1143 - f1_m: 9.0498e-04 - precision_m: 0.0294 - recall_m: 4.5956e-04